#  R&D Capital Replication


| Name | EID |
| --- | --- |
| Brandt Green | bwg537 |

In [369]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
import statsmodels.formula.api as smf
import datetime
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.float_format', lambda x: '%.5f' % x)

## Exract Data
Data for this assignment comes from 3 sources:
* Security Fundamental Data: [CRSP/Compustat Merged](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/crspcompustat-merged/fundamentals-annual/)
* Security Price/Return Data: [CRSP Monthly](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/monthly-stock-file/)
* Market Variables: [Ken French](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)

First, we have the market data from Ken French's website. Below, we read in this data set and make a few small adjustments to get the dataframe in a form that will be easy to handle when we need it later on.

In [370]:
french_df = pd.read_csv('french.csv')

# Convert date to datetime and add new columns for year and month.
french_df['date'] = pd.to_datetime(french_df.date,format='%Y%m')
french_df = french_df.set_index('date')
french_df['year'] = french_df.index.year
french_df['month'] = french_df.index.month

# Making this dataframe multi-indexed by year and month makes working with the other return dataframes later on much easier
french_df = french_df.set_index(keys=['year','month']) 

# Convert the data from percents to decimal representation
french_df = french_df/100 
french_df

Mkt-RF      SMB      HML      RF
year month                                   
1926 7      0.02960 -0.02380 -0.02730 0.00220
     8      0.02640 -0.01470  0.04140 0.00250
     9      0.00360 -0.01390  0.00120 0.00230
     10    -0.03240 -0.00130  0.00650 0.00320
     11     0.02530 -0.00160 -0.00380 0.00310
...             ...      ...      ...     ...
2021 5      0.00290 -0.00280  0.07040 0.00000
     6      0.02750  0.01800 -0.07760 0.00000
     7      0.01270 -0.03960 -0.01750 0.00000
     8      0.02900 -0.00480 -0.00130 0.00000
     9     -0.04370  0.00800  0.05090 0.00000

[1143 rows x 4 columns]

Now we read in the security data. The data is in two separate files which we will clean separately. The fundamental data, which contains historical R&D numbers is in the crsp/compustat merged file, which will have one row for each security for each year there is data available. The security price data is in the crsp file, which will have a row for each month for each security.

In [371]:
ccm = pd.read_csv('ccm_merged.csv')
crsp = pd.read_csv('crsp.csv')

We start by cleaning the fundamental data first. We are primarily just filtering the data to ignore data older than our time period and to ensure we are only examining U.S publically traded stocks. The most import 'cleaning' that is done is the date shifting. We take our 'date' variable and shift it into the future by 90 days to create the relevant date variable which should now represent the new date that we the data would have been available in realtime. 

In [372]:
# The csv file has a bunch of columns that we don't actually need, so create a new df with the appropriate columns
cols_to_keep = ['LINKTYPE', 'LPERMNO', 'datadate', 'tic', 'conm', 'curcd', 'xrd', 'fic', 'sich', 'sic','exchg']
fund_df = ccm.copy()[cols_to_keep]

# Convert date variable to datetime 
fund_df['datadate'] = pd.to_datetime(fund_df['datadate'],format='%Y%m%d')
# fund_df['year'] = fund_df.date_real.dt.year

# Ignore dates before 1975. Since our portfolio return calculations start in 1981, we don't need the super old data.
fund_df = fund_df[fund_df['datadate'].dt.year >= 1974]

# Shift the data by 3 months to account for the reporting lag in real time
fund_df['date_real'] = fund_df['datadate'] + pd.DateOffset(90)


# Filter for proper links.  I don't know if this is actually necessary but WRDS said you should
fund_df = fund_df[fund_df['LINKTYPE'].isin(['LU','LC'])] 

# This should filter to make sure we only examine US companies.
fund_df = fund_df[fund_df['curcd'] == 'USD'] 
fund_df = fund_df[fund_df['fic'] == 'USA'] 


# Replace historically missing SIC codes with the current one
fund_df['sich'] = fund_df['sich'].fillna(fund_df['sic'])

# Clean up sich codes to make sure there are no weird strings and stuff
fund_df['sich'] = fund_df.sich.astype(float).astype(int) 

# I believe this line is to filter out financial companies
fund_df = fund_df[~((fund_df['sich'] >= 6000 ) & (fund_df['sich'] <=6999))]

# Filter for the correct exchanges. (I think this is just publically traded exchanges)
fund_df = fund_df[(fund_df.exchg >= 10) & (fund_df.exchg <=19)]

# If R&D is NA, set it to 0. 
fund_df = fund_df.fillna(value={'xrd':0})

# Sort by permno and date
fund_df = fund_df.sort_values(['LPERMNO','date_real'])

# If there wasn't an R&D previosuly then we set it to zero.

# fund_df = fund_df.drop(columns=['LINKTYPE','curcd','fic','conm','xrd']) exchg
fund_df = fund_df.drop(columns=['LINKTYPE','curcd','fic','conm','sic','datadate','sich','exchg'])

fund_df

,LPERMNO,tic,xrd,date_real
153225,10000,OMFGA,0.03900,1987-01-29
153130,10001,EGAS,0.00000,1986-09-28
153131,10001,EGAS,0.00000,1987-09-28
153132,10001,EGAS,0.00000,1988-09-28
153133,10001,EGAS,0.00000,1989-09-28
...,...,...,...,...
298864,93436,TSLA,834.40800,2017-03-31
298865,93436,TSLA,1378.07300,2018-03-31
298866,93436,TSLA,1473.63400,2019-03-31
298867,93436,TSLA,1390.00000,2020-03-30


We clean the pricing data similar to above. 

In [373]:
crsp_cols_to_keep = ['PERMNO', 'date', 'SHRCD', 'TICKER', 'COMNAM', 'SHRCLS', 'PRC', 'RET', 'SHROUT']
return_df = crsp.copy()[crsp_cols_to_keep]


return_df['date'] = pd.to_datetime(return_df['date'],format='%Y%m%d')
# Ignore dates before 1974. Since our portfolio return calculations start in 1981, we don't need the super old data.
return_df = return_df[return_df['date'].dt.year >= 1974]

# Only filter for Common stock securities. 11,10 cat codes are for like 'normal' securities
return_df = return_df[return_df.SHRCD.isin([11,10])] 

# We only want share class 'A', which is common stocks only.
# Also, we keep the NAs. Not exactly sure why, but I believe it's correct. I beleive TSLA was showing up as NA for this.
return_df = return_df[(return_df.SHRCLS == 'A') | (return_df.SHRCLS.isna())]

# Shares outstanding is originally expressed in 1000s. So we convert to dollars
return_df['SHROUT'] = return_df['SHROUT'] * 1000

# Convert any negative values in prc to positives. The negatives just indiciate that PRC is average of the bid ask spread instead of closing price.
return_df['PRC'] = return_df['PRC'].abs()

# Calculate Market cap
return_df['market_cap'] = return_df.PRC * return_df.SHROUT

# Ignore companies that are not also present in our fundamentals database.
# Not sure if this is correct, but I think so? They should be the same right? Only difference should be based on different filterings for each data set
return_df = return_df[return_df.PERMNO.isin(fund_df.LPERMNO.unique())]

# Filter out weird returns that have letters in them. I believe 'C' indicates it's the first data date
return_df = return_df[~return_df.RET.astype(str).str.contains('[a-zA-Z]')].copy() 
return_df.RET = return_df.RET.astype(float)

# Add year and month columns
return_df['year'] = return_df['date'].dt.year
return_df['month'] = return_df.date.dt.month

return_df = return_df.drop(columns=['SHRCD','SHRCLS','PRC','SHROUT','COMNAM','TICKER'])

return_df

,PERMNO,date,RET,market_cap,year,month
2,10000,1986-02-28,-0.25714,11960000.00000,1986,2
3,10000,1986-03-31,0.36539,16330000.00000,1986,3
4,10000,1986-04-30,-0.09859,15172000.00000,1986,4
5,10000,1986-05-30,-0.22266,11793878.34000,1986,5
6,10000,1986-06-30,-0.00502,11734593.75000,1986,6
...,...,...,...,...,...,...
4118752,93436,2020-08-31,0.74145,464339070198.09003,2020,8
4118753,93436,2020-09-30,-0.13909,406701489480.00000,2020,9
4118754,93436,2020-10-30,-0.09550,367823513519.01001,2020,10
4118755,93436,2020-11-30,0.46274,538028588641.97998,2020,11


Below we create two important new dataframes by pivoting the return_df. Both of these pivot tables will just make it very efficient for us to lookup a securities market cap on return for a particular month, which we will need to do later.

In [374]:
market_cap_piv = pd.pivot_table(data=return_df,columns='PERMNO',index=['year','month'],values='market_cap')
returns_piv = pd.pivot_table(data=return_df,columns='PERMNO',index=['year','month'],values='RET')
market_cap_piv

PERMNO      10000  10001  10005  10007  10008  10010  10011  10012  \
year month                                                           
1974 1        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     2        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     3        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     4        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     5        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...           ...    ...    ...    ...    ...    ...    ...    ...   
2020 8        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     9        NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     10       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     11       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
     12       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

PERMNO              10014  10015  ...           93422            93423  \
year month                        ...                                    
1974 1     11602500.00000    NaN  ...             NaN              NaN   
     2     10237500.00000    NaN  ...             NaN              NaN   
     3     11602500.00000    NaN  ...             NaN              NaN   
     4      9555000.00000    NaN  ...             NaN              NaN   
     5      8190000.00000    NaN  ...             NaN              NaN   
...                   ...    ...  ...             ...              ...   
2020 8                NaN    NaN  ... 315022500.00000 1841878260.00000   
     9                NaN    NaN  ... 218771010.00000 1720668600.00000   
     10               NaN    NaN  ... 217998900.00000 1837202740.00000   
     11               NaN    NaN  ... 389975810.00000 2611343210.00000   
     12               NaN    NaN  ... 580617040.00000 2897715700.00000   

PERMNO               93426  93428  93430  93432  93433          93434  93435  \
year month                                                                     
1974 1                 NaN    NaN    NaN    NaN    NaN            NaN    NaN   
     2                 NaN    NaN    NaN    NaN    NaN            NaN    NaN   
     3                 NaN    NaN    NaN    NaN    NaN            NaN    NaN   
     4                 NaN    NaN    NaN    NaN    NaN            NaN    NaN   
     5                 NaN    NaN    NaN    NaN    NaN            NaN    NaN   
...                    ...    ...    ...    ...    ...            ...    ...   
2020 8     312670320.00000    NaN    NaN    NaN    NaN 78567550.00000    NaN   
     9     317816640.00000    NaN    NaN    NaN    NaN 82958480.00000    NaN   
     10    299867280.00000    NaN    NaN    NaN    NaN 76307040.00000    NaN   
     11    367146000.00000    NaN    NaN    NaN    NaN 87388020.00000    NaN   
     12    395136960.00000    NaN    NaN    NaN    NaN 98102260.00000    NaN   

PERMNO                  93436  
year month                     
1974 1                    NaN  
     2                    NaN  
     3                    NaN  
     4                    NaN  
     5                    NaN  
...                       ...  
2020 8     464339070198.09003  
     9     406701489480.00000  
     10    367823513519.01001  
     11    538028588641.97998  
     12    668905279711.97998  

[564 rows x 16632 columns]

## Calculate Returns
We've loaded and cleaned all of the data we need, so now we can start forming portfolios!

We create 3 different dataframes which we will use to store all of the return information for all of our portfolios. For now it will contain only NAN values, but we will fill it in momentarily. 

In [375]:
# port_names is an important variable that will be referenced a lot!
port_names = [f'p_{i}' for i in range(1,6)] + ['p_no_rnd']

value_weighted_ports_df = pd.DataFrame(index=returns_piv.index,columns=port_names)
equal_weight_ports_df = pd.DataFrame(index=returns_piv.index,columns=port_names)
small_firm_value_weighted_ports_df = pd.DataFrame(index=returns_piv.index,columns=port_names)
value_weighted_ports_df

p_1  p_2  p_3  p_4  p_5 p_no_rnd
year month                                  
1974 1      NaN  NaN  NaN  NaN  NaN      NaN
     2      NaN  NaN  NaN  NaN  NaN      NaN
     3      NaN  NaN  NaN  NaN  NaN      NaN
     4      NaN  NaN  NaN  NaN  NaN      NaN
     5      NaN  NaN  NaN  NaN  NaN      NaN
...         ...  ...  ...  ...  ...      ...
2020 8      NaN  NaN  NaN  NaN  NaN      NaN
     9      NaN  NaN  NaN  NaN  NaN      NaN
     10     NaN  NaN  NaN  NaN  NaN      NaN
     11     NaN  NaN  NaN  NaN  NaN      NaN
     12     NaN  NaN  NaN  NaN  NaN      NaN

[564 rows x 6 columns]

### Helper Functions
Here are just a few helper functions that we will need in a bit.

In [376]:
# Based on formula given in power point slide
RND_WEIGHTS = np.array([1,.8,.6,.4,.2])

# These are lables that we feed in to pandas qcut in get_sorting_var below.
QCUT_LABELS = [f'p_{i}' for i in range(1,6)] 


def get_currently_available_permnos(year:int,month:int) -> list:
    """Return a list of all the permnos where there was a valid return during this month. (Non-NA) returns based on the returns_piv."""
    return returns_piv.loc[(year,month)].dropna().index.to_list()
    

def get_sorting_var(date:datetime.date, permno:int):
    """Send in a date, permno combo and get the rdc/market_cap value for the company as it would be calcualted on that date."""

    # Filter the fundamental df for company that we want and only look at dates prior to this one.
    company_xrds = fund_df[(fund_df.LPERMNO == permno) & (fund_df.date_real <= date)].sort_values('date_real',ascending=False)['xrd']
    
    # Loop through the dates by most recent and fill them into an array up to the 5 most recent
    num_dates = len(company_xrds)
    rnds = np.zeros(5) # Default is that R&D number is 0, unless we find data to fill in
    
    for i in range(min(num_dates,5)):
        rnds[i] = company_xrds.iloc[i]

    # Calculate the weighted average (Not sure if this is actually considered a weighted average or what since it doesn't sum to 1)
    rdc = RND_WEIGHTS.dot(rnds) 

    # Get last available market cap
    market_cap = market_cap_piv.loc[(date.year,date.month),permno]

    return rdc/market_cap


def get_sorted_var_series_small(sorted_var_series:pd.Series, trade_date) ->pd.Series:
    """We take a sorted_variable series and strip out the largest 1000 companies based on market cap."""

    # Put the info all in one data frame so you can sort it together.
    df = pd.DataFrame(sorted_var_series,columns=['sorted_var'])
    df['market_cap'] = market_cap_piv.loc[(trade_date.year,trade_date.month),df.index]
    df = df.sort_values('market_cap',ascending=False).iloc[1000:] # Exclude the 1000 biggest
    return df['sorted_var']



def create_new_sorted_ports(sorted_var:pd.Series) -> dict:
    """Returns a dictionary where keys are port_names and values are a list of permnos to be included in that portfolio."""

    port_dict:dict = {}
    non_zero_rnd_perms = sorted_var[sorted_var > 0]
    quartile_cuts = pd.qcut(non_zero_rnd_perms,5,labels=QCUT_LABELS)

    for cut_label in QCUT_LABELS:
        port_dict[cut_label] = quartile_cuts[quartile_cuts == cut_label].index.to_list()

    port_dict['p_no_rnd'] = sorted_var[sorted_var == 0].index.to_list()

    return port_dict

def calculate_port_return(permnos:list, year, month, method='equal_weight') -> float:
    """Send in a list of permnos along with the relevant year, month combo and this will calculate what the return is for that period."""

    # Only look at companies with actual returns in the month
    valid_returns = returns_piv.loc[(year,month)][permnos].dropna() 

    # Weights will be a series where the index is the permno and the values are the weights.
    if method == 'equal_weight':
        weights = pd.Series(index=valid_returns.index, data=np.full(shape=len(valid_returns),fill_value=1/len(valid_returns)))
    elif method == 'value_weight':
        # Get the market caps by using the end of previous month value. 
        if month == 1:
            cap_lookup_index = (year-1,12)
        else:
            cap_lookup_index = (year, month-1)

        valid_market_caps = market_cap_piv.loc[cap_lookup_index][valid_returns.index] # Filter for valid permnos only

        # Market cap weights
        weights = valid_market_caps/valid_market_caps.sum() 
    
    monthly_return = (weights * valid_returns).sum()

    return monthly_return

## Calculate Returns
This code block is where most of the magic happens.

WARNING!! Below cell takes like 4-5 minutes to run!!!

In [377]:

# Calculate returns for all of the years between 1981-2020, inclusive

for year in list(range(1981,2021)):

    # 'trading_date' is meant to indicate when we are reconstituting the portfolio. 
    # We need this variable so that we can filter the dataframes to only look at data prior to the trading date

    trade_date = pd.to_datetime((f'{year-1}-12-31'))

    # Get a list of permnos containing all the the stocks available to us on this date.
    # Available stocks is determined by looking at stocks that were trading at the end of the previous year
    available_stock_permnos =  get_currently_available_permnos(year-1,12) 
    
    # Here we get a series where the index is all of the available permnos and the values will be the RDC/ME as determined on this trading date
    sorted_var_series = pd.Series(index=available_stock_permnos,data=[get_sorting_var(trade_date, permy) for permy in available_stock_permnos]).dropna()

    # We get another similar series, that excludes the 1000 biggest firms.
    sorted_var_series_small = get_sorted_var_series_small(sorted_var_series,trade_date)

    # We create a portfolio dict where the dictionary keys are the portfolio names and the values are lists of the permnos included in that portfolio
    portfolio_perms_dict = create_new_sorted_ports(sorted_var_series)
    
    # Same as above, but for the small companies
    portfolio_perms_dict_small = create_new_sorted_ports(sorted_var_series_small)
    
    # Now loop through each month to calculate the monthly returns for all the portfolios.
    for month in range(1,13):

        for port_name, permnos in portfolio_perms_dict.items():
            # Calculate equal weighted return and value weighted returns
            equal_weight_ports_df.loc[(year,month),port_name] = calculate_port_return(permnos,year,month,method='equal_weight')
            value_weighted_ports_df.loc[(year,month),port_name] = calculate_port_return(permnos,year,month,method='value_weight')

        for port_name, permnos in portfolio_perms_dict_small.items():
            # Calculate value weighted returns for the small-firm portfolios
            small_firm_value_weighted_ports_df.loc[(year,month),port_name] = calculate_port_return(permnos,year,month,method='value_weight') 

We need to clean up the portfolio return dfs a bit, but then we are good to go. We've got return data for all of the portfolios we need.

In [378]:
# clean up
equal_weight_ports_df = equal_weight_ports_df.dropna()
value_weighted_ports_df = value_weighted_ports_df.dropna()
small_firm_value_weighted_ports_df = small_firm_value_weighted_ports_df.dropna()
value_weighted_ports_df

p_1      p_2      p_3      p_4      p_5 p_no_rnd
year month                                                      
1981 1     -0.10288 -0.02639 -0.04929 -0.03078 -0.02327 -0.02630
     2      0.01653 -0.00608  0.03435  0.05197  0.03291  0.00700
     3      0.01094  0.03131  0.05922  0.02886  0.05139  0.06863
     4     -0.07453  0.00970  0.01514 -0.01184  0.00824 -0.01314
     5     -0.02891  0.01296  0.02243  0.01065  0.00757  0.01274
...             ...      ...      ...      ...      ...      ...
2020 8      0.04295  0.14815  0.05350  0.02869  0.01782  0.06083
     9     -0.03163 -0.06317 -0.04453 -0.02596 -0.01899 -0.00571
     10    -0.01623 -0.03458 -0.02400 -0.03554  0.05293 -0.01852
     11     0.10247  0.11542  0.10583  0.14373  0.24306  0.12949
     12     0.04007  0.06171  0.04331  0.06456 -0.01026  0.02755

[480 rows x 6 columns]

## 1) Quintile Sorted Returns by period

Calculate excess returns by subtracting out risk free rate for each month:

In [379]:
equal_weight_rp = equal_weight_ports_df.sub(french_df.RF, axis=0).dropna()
value_weight_rp = value_weighted_ports_df.sub(french_df.RF, axis=0).dropna()
small_value_weight_rp = small_firm_value_weighted_ports_df.sub(french_df.RF, axis=0).dropna()

Create dataframes to store the results for each portfolio and for each period.

In [316]:
sample_periods = {'1981.07-2012.12':[(1981,7),(2012,12)],'1981.07-1999.12':[(1981,7),(1999,12)],'2000.01-2012.12':[(2000,1),(2012,12)],'1981.07-2020.12':[(1981,7),(2020,12)]}

equal_weight_results = pd.DataFrame(columns=port_names,index=sample_periods.keys())
value_weight_results = pd.DataFrame(columns=port_names,index=sample_periods.keys())
small_firm_value_weight_results = pd.DataFrame(columns=port_names,index=sample_periods.keys())

equal_weight_results.index.name = 'Period'
value_weight_results.index.name = 'Period'
small_firm_value_weight_results.index.name = 'Period'

Calcualte mean excess returns by portfolio and put the results into the dataframes.

In [381]:
for sample_period,dates in sample_periods.items():
    equal_weight_results.loc[sample_period] = equal_weight_rp.loc[dates[0]:dates[1]].mean()
    value_weight_results.loc[sample_period] = value_weight_rp.loc[dates[0]:dates[1]].mean()
    small_firm_value_weight_results.loc[sample_period] = small_value_weight_rp.loc[dates[0]:dates[1]].mean()

equal_weight_results = equal_weight_results * 100
value_weight_results = value_weight_results * 100
small_firm_value_weight_results = small_firm_value_weight_results * 100

### Equal Weight Results

In [382]:
equal_weight_results

,p_1,p_2,p_3,p_4,p_5,p_no_rnd
Period,,,,,,
1981.07-2012.12,0.19648,0.56447,0.82975,1.18347,2.16181,0.62349
1981.07-1999.12,0.17105,0.50917,0.81411,1.23803,2.10059,0.50361
2000.01-2012.12,0.23267,0.64317,0.85199,1.10584,2.24893,0.79409
1981.07-2020.12,0.38166,0.72388,0.98861,1.25805,2.10659,0.69128


### Value Weighted Results

In [383]:
value_weight_results

,p_1,p_2,p_3,p_4,p_5,p_no_rnd
Period,,,,,,
1981.07-2012.12,0.44870,0.66441,0.68695,1.11203,1.01474,0.52706
1981.07-1999.12,0.79014,0.98134,0.94048,1.30042,1.24095,0.76704
2000.01-2012.12,-0.03719,0.21340,0.32616,0.84394,0.69281,0.18556
1981.07-2020.12,0.55193,0.86048,0.85027,1.21100,1.16050,0.61440


### Value-Weighted Results, Exluding Biggest 1000 Firms

In [384]:
small_firm_value_weight_results

,p_1,p_2,p_3,p_4,p_5,p_no_rnd
Period,,,,,,
1981.07-2012.12,0.21430,0.63160,0.73109,0.92182,1.73983,0.50880
1981.07-1999.12,0.22952,0.64215,0.76600,0.92155,1.55970,0.43782
2000.01-2012.12,0.19263,0.61659,0.68142,0.92220,1.99616,0.60982
1981.07-2020.12,0.39676,0.83407,0.95088,1.14964,1.83459,0.59414


## Long-Short Portfolios

Get Excess returns for long short portfolios and put the information into one dataframe that we can use to run our regressions

In [386]:
regression_df = french_df.copy()
regression_df['LS_equal'] = (equal_weight_ports_df.p_5 - equal_weight_ports_df.p_1).astype(float).sub(french_df.RF, axis=0).dropna()
regression_df['LS_value'] = (value_weighted_ports_df.p_5 - value_weighted_ports_df.p_1).astype(float).sub(french_df.RF, axis=0).dropna()
regression_df['LS_small_value'] = (small_firm_value_weighted_ports_df.p_5 - small_firm_value_weighted_ports_df.p_1).astype(float).sub(french_df.RF, axis=0).dropna()
regression_df = regression_df.dropna()
regression_df

Mkt-RF      SMB      HML      RF  LS_equal  LS_value  \
year month                                                          
1981 1     -0.05040  0.02920  0.06720 0.01040   0.03043   0.06921   
     2      0.00570 -0.00340  0.01020 0.01070  -0.00119   0.00569   
     3      0.03560  0.03540  0.00640 0.01210   0.00030   0.02835   
     4     -0.02110  0.04400  0.02280 0.01080   0.00789   0.07197   
     5      0.00110  0.02000 -0.00420 0.01150  -0.00893   0.02499   
...             ...      ...      ...     ...       ...       ...   
2020 8      0.07630 -0.00260 -0.02920 0.00010  -0.08119  -0.02522   
     9     -0.03630  0.00020 -0.02620 0.00010  -0.00642   0.01254   
     10    -0.02100  0.04410  0.04160 0.00010  -0.04524   0.06906   
     11     0.12470  0.05650  0.02130 0.00010   0.08903   0.14049   
     12     0.04630  0.04820 -0.01430 0.00010   0.04358  -0.05043   

            LS_small_value  
year month                  
1981 1             0.01283  
     2             0.01599  
     3             0.01503  
     4            -0.02267  
     5            -0.03634  
...                    ...  
2020 8            -0.18222  
     9            -0.04999  
     10           -0.05420  
     11            0.11991  
     12           -0.12551  

[480 rows x 7 columns]

### Regressions, alphas, and Sharpe Ratios

In [395]:
ls_port_names = ['LS_equal','LS_value','LS_small_value']
ls_results = pd.DataFrame(columns=['CAPM_Alpha','CAPM_t_stat', 'FamaFrench_Alpha','FamaFrench_t_stat','Sharpe_Ratio'],index=ls_port_names)

for port in ls_port_names:
    capm_model = smf.ols(formula=f"{port} ~ Q('Mkt-RF')", data=regression_df)
    capm_result = capm_model.fit()
    ls_results.loc[port,'CAPM_Alpha'] = capm_result.params['Intercept']
    ls_results.loc[port,'CAPM_t_stat'] = capm_result.tvalues['Intercept']

    ff_model = smf.ols(formula=f"{port} ~ Q('Mkt-RF') + SMB + HML", data=regression_df)
    ff_result = ff_model.fit()
    ls_results.loc[port,'FamaFrench_Alpha'] = ff_result.params['Intercept']
    ls_results.loc[port,'FamaFrench_t_stat'] = ff_result.tvalues['Intercept']

    ls_results.loc[port,'Sharpe_Ratio'] = regression_df[port].mean()/regression_df[port].std()


ls_results[['CAPM_Alpha','FamaFrench_Alpha']] = ls_results[['CAPM_Alpha','FamaFrench_Alpha']]*100
ls_results

,CAPM_Alpha,CAPM_t_stat,FamaFrench_Alpha,FamaFrench_t_stat,Sharpe_Ratio
LS_equal,1.26955,4.84211,1.27863,5.20449,0.24434
LS_value,0.06656,0.28785,-0.11528,-0.55151,0.06099
LS_small_value,0.94485,3.83547,0.97535,4.09599,0.20373
